# 1. VideoFrameGenerator

#### Class(Fixed)
- Normal : 150
- Arson (+ Explosion) : 50 + 50 = 100
- Assault (+ Abuse, Fighting) : 50 + 50 + 50 = 150
- Burglary : 100

#### Image Size(Fixed)
- 128 X 128

#### \# of Frames(Fixed)
- 64

#### Data Split(Fixed)
- 7 : 2 : 1

#### Color Scale
- **Gray** or RGB

#### Frame Generator
- Basic or Sliding or **OpticalFlow**

In [6]:
MODELNAME='g_optical_conv_gru'

SIZE = (128, 128)
CHANNELS = 1 # Gray scale
NBFRAME = 64 
BS = 16
SHUFFLE = True

SPLIT_RATIO = (.2, .1)

In [7]:
import os
import glob
import keras

import keras_video 
from keras_video import OpticalFlowGenerator

classes = [i.split(os.path.sep)[1] for i in glob.glob('zip_integrate/*')]
classes.sort()

# pattern to get videos and classes
glob_pattern='zip_integrate/{classname}/*.mp4'

# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)

# Create sliding frame generator
train = OpticalFlowGenerator(
    method= keras_video.METHOD_ABS_DIFF, # optical flow method
    classes=classes, # class list
    glob_pattern=glob_pattern, # directory path
    nb_frames=NBFRAME, # #of frames to return for each sequence
    rescale=1/255., # normalization
    split_val=.2, # split validation
    split_test=.1, # split test
    shuffle=True, # randomize
    batch_size=BS, # batch size
    target_shape=SIZE, # image size
    nb_channel=CHANNELS, # gray scale
    transformation=data_aug, # data augmentation
    use_frame_cache=False)
    
valid = train.get_validation_generator()
test = train.get_test_generator()

class Arson, validation count: 20, test count: 8, train count: 72
class Assault, validation count: 30, test count: 12, train count: 108
class Burglary, validation count: 20, test count: 8, train count: 72
class Normal_Videos-Part-1, validation count: 30, test count: 12, train count: 108
Total data: 4 classes for 360 files for train
Total data: 4 classes for 100 files for validation
Total data: 4 classes for 40 files for test


In [8]:
import matplotlib.pyplot as plt
import keras_video.utils

# keras_video.utils.show_sample(test, index=0, random=False, row_width=10, row_height=10)

# 2. Modeling

#### Conv2d + GRU

In [9]:
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalAveragePooling2D

def build_convnet(shape=(128, 128, 3)):
    momentum = .9

    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape, padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalAveragePooling2D())
    return model

In [10]:
from keras.layers import TimeDistributed, GRU, Dense, Dropout

def action_model(shape=(64, 128, 128, 3), nbout=4):
    # Create our convnet with (128, 128, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (64, 128, 128, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [11]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (64, 128, 128, 3)

model = action_model(INSHAPE, len(classes))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 64, 512)           1553664   
_________________________________________________________________
gru (GRU)                    (None, 64)                110976    
_________________________________________________________________
dense (Dense)                (None, 1024)              66560     
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)              

In [12]:
optimizer = keras.optimizers.Adam(0.01)

model.compile(optimizer,
              'categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
EPOCHS=50
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside

callbacks = [keras.callbacks.ReduceLROnPlateau(verbose=1),
             keras.callbacks.ModelCheckpoint(
             'chkp/weights_' + MODELNAME + '_gray_basic_conv_gru.{epoch:02d}-{val_loss:.2f}.hdf5',
             verbose=1),]

In [ ]:
%%time

hist_model = model.fit(train,
             validation_data=valid,
             verbose=1,
             epochs=EPOCHS,
             callbacks=callbacks)

Epoch 1/50


In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(hist_model.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, hist_model.history['loss'])
plt.plot(epochs, hist_model.history['val_loss'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Training Loss', 'Validation Loss'])
plt.grid()
plt.show()

In [ ]:
epochs = range(1, len(hist_model.history['loss']) + 1)

plt.figure(figsize = (9, 6))
plt.plot(epochs, hist_model.history['accuracy'])
plt.plot(epochs, hist_model.history['val_accuracy'])

plt.title('Training & Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training Accuracy', 'Validation Accuracy'])
plt.grid()
plt.show()

loss, accuracy = model.evaluate(test)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))

In [ ]:
save_dir = 'Models'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

final_model_name = '/' + MODELNAME + '.h5'

model.save(save_dir + final_model_name)

In [ ]:
from keras.models import load_model

lmn = load_model(save_dir + final_model_name)

loss, accuracy = lmn.evaluate(test)

print('Loss = {:.5f}'.format(loss))
print('Accuracy = {:.5f}'.format(accuracy))